In [1]:
%load_ext autoreload

In [92]:
%autoreload 2
from torchdrive.data import collate
from torchdrive.datasets.rice import MultiCamDataset
from torchdrive.notebook import display_img, display_color, display, to_pil_image

a = MultiCamDataset(
    index_file="../../openape/snapshots/out-mar23/index.txt",
    mask_dir="../../openape/masks",
    cameras=["main", "narrow", "fisheye", "leftpillar", "leftrepeater", "rightpillar", "rightrepeater", "backup"],
    cam_shape=(480, 640),
    nframes_per_point=5,
    limit_size=1000,
    dynamic=True,
)
print(len(a))
example = a[800]
batch = collate([example])

#for cam in a.cameras:
#    display_img(example.color[cam][0].float())

heading_weights {5: 0.5555555555555556, 45: 1.6666666666666667, 15: 1.6666666666666667}


/home/rice/venvs/openape/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


1196


In [93]:
import math
import torch
from pythreejs import *
from IPython.display import display

SCALE = 3
D = 256/SCALE
W = 256/SCALE
H = 12/SCALE

view_width = 600
view_height = 400
camera = PerspectiveCamera( position=[-10, 6, 10], aspect=640/view_height)
camera.up = (0, 0, 1)
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight()

grid_helper1 = GridHelper(20, 20, '#888', '#444')
grid_helper10 = GridHelper(100, 10, '#888', '#444')
grid_helper1.rotateX(math.pi/2)
grid_helper10.rotateX(math.pi/2)

scene = Scene(children=[grid_helper1, grid_helper10, camera, key_light, ambient_light], background='#111')
frame = 0

for cam in batch.T:
    x = torch.tensor((
        (0, 0, 0, 1.0),
        (1, 0, 0, 1.0),
        (0, 1, 0, 1.0),
        (0, 0, 1, 1.0),
    ))
    T = batch.cam_to_world(cam, frame)
    pos = T.matmul(x.T).T
    geo = AxesHelper(1)
    geo.up = (0, 0, 1)
    geo.matrixAutoUpdate = False
    geo.matrix = tuple(T.T.contiguous().view(-1).tolist())
    scene.add([geo])

#sphere.position = (0, 0, 0)
renderer = Renderer(camera=camera, scene=scene, controls=[OrbitControls(controlling=camera)], width=view_width, height=view_height)
display(renderer)

Renderer(camera=PerspectiveCamera(aspect=1.6, position=(-10.0, 6.0, 10.0), projectionMatrix=(1.0, 0.0, 0.0, 0.…